# Group E

***

<div class="alert alert-block alert-info">
    
# INTRODUCTION

As the price of tickets is decreasing, people are travelling more for business and pleasure. Moreover, the supply chains are linked between countries and continents by airplanes, boats, cars etc.. So the increase in demand for mobility leads to a high dependence on transportation systems. When planes are delayed because of several different reasons, many people get late to business meetings, products don’t arrive on time and all of these lead to additional costs and sometimes loss in profit for companies.

<br>
    
An analysis will be done on a dataset of January 2008 flights in USA by merging weather variables of each airport depending on location and day to see if weather is an important factor for when flights are delayed or not.
    
    Outline
        I- Read the Data and Prepare the Dataset
        II- Analyze the Data
        III- Answer the Questions
    
</div>

### ENVIRONMENT SETUP

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


from pyspark.sql import functions as F
from pyspark.sql import types as T
from IPython.display import display, Markdown

<br>

### READ DATA

**Reading flights data**

In [2]:
flightsDF = spark.read\
                 .option("header", "true")\
                 .csv("flights_jan08.csv")


# selecting only variables needed for the analysis
flightsDF = flightsDF.select(["Month", "DayofMonth", "DayOfWeek", "ArrDelay", "DepDelay", 
                              "Origin", "Dest", "Cancelled", "CancellationCode", "Diverted", "WeatherDelay"])


flightsDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % flightsDF.count()))


root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)



This DataFrame has **100000 rows**.

**Reading weather info files**

<div>

https://www.ncdc.noaa.gov/orders/qclcd/
                                                   
    We downloaded this zip file: QCLCD200801.zip to have weather conditions of the airports in January 2008. From the zip file, we used the daily(weatherDF), and station (airportDF) datasets.

</div>

Attached with the assignment is the zip file containing two csv files in case the link did not work

In [3]:
# csv containing weather info and area codes

weatherDF = spark.read\
                 .option("header", "true")\
                 .csv("Weather per day per station.csv")

weatherDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % weatherDF.count()))

In [4]:
# csv containing a mapping of area code to airport name

airportDF = spark.read\
                 .option("header", "true")\
                 .csv("Airport Code.csv")

airportDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % airportDF.count()))

root
 |-- WBAN: string (nullable = true)
 |-- WMO: string (nullable = true)
 |-- CallSign: string (nullable = true)
 |-- ClimateDivisionCode: string (nullable = true)
 |-- ClimateDivisionStateCode: string (nullable = true)
 |-- ClimateDivisionStationCode: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- GroundHeight: string (nullable = true)
 |-- StationHeight: string (nullable = true)
 |-- Barometer: string (nullable = true)
 |-- TimeZone: string (nullable = true)



This DataFrame has **2279 rows**.

In [5]:
# joining both files to finally have a common variable with main dataset January2008

joinedDF = weatherDF.join(airportDF, on = "WBAN")\
                    .select(weatherDF["*"],
                            airportDF["CallSign"].alias("AirportCode"))

joinedDF.cache()

DataFrame[WBAN: string, YearMonthDay: string, Tmax: string, TmaxFlag: string, Tmin: string, TminFlag: string, Tavg: string, TavgFlag: string, Depart: string, DepartFlag: string, DewPoint: string, DewPointFlag: string, WetBulb: string, WetBulbFlag: string, Heat: string, HeatFlag: string, Cool: string, CoolFlag: string, Sunrise: string, SunriseFlag: string, Sunset: string, SunsetFlag: string, CodeSum: string, CodeSumFlag: string, Depth: string, DepthFlag: string, Water1: string, Water1Flag: string, SnowFall: string, SnowFallFlag: string, PrecipTotal: string, PrecipTotalFlag: string, StnPressure: string, StnPressureFlag: string, SeaLevel: string, SeaLevelFlag: string, ResultSpeed: string, ResultSpeedFlag: string, ResultDir: string, ResultDirFlag: string, AvgSpeed: string, AvgSpeedFlag: string, Max5Speed: string, Max5SpeedFlag: string, Max5Dir: string, Max5DirFlag: string, Max2Speed: string, Max2SpeedFlag: string, Max2Dir: string, Max2DirFlag: string, AirportCode: string]

**Joining all files**

In [6]:
# selecting features needed to merge with flightsDF

filteredDF = joinedDF.select("AirportCode",
                             F.substring(F.col("YearMonthDay"), 7, 2).alias("Day"),
                             F.col("Tavg").alias("AvgTemp"),
                             F.col("AvgSpeed").alias("AvgWindSpeed"),
                             F.col("codeSum").alias("WeatherCode"))

filteredDF.cache()

DataFrame[AirportCode: string, Day: string, AvgTemp: string, AvgWindSpeed: string, WeatherCode: string]

In [7]:
# merging with flightsDF on "Origin"

joinedDF = flightsDF.join(filteredDF,
                          on = ((flightsDF["Origin"] == filteredDF["AirportCode"]) &
                                (flightsDF["DayofMonth"] == filteredDF["Day"])),
                          how = 'left')

joinedDF = joinedDF.drop("Day").drop("AirportCode")


joinedDF = joinedDF.withColumnRenamed("AvgTemp", "AvgTempOrigin")\
                   .withColumnRenamed("WeatherCode", "WeatherCodeOrigin")\
                   .withColumnRenamed("AvgWindSpeed", "AvgWindSpeedOrigin")


In [8]:
# merging with flightsDF on "Dest"

joinedDF = joinedDF.join(filteredDF,
                         on = ((joinedDF["Dest"] == filteredDF["AirportCode"]) & 
                               (joinedDF["DayofMonth"] == filteredDF["Day"])),
                         how = 'left')


joinedDF = joinedDF.drop("Day").drop("AirportCode")


joinedDF = joinedDF.withColumnRenamed("AvgTemp", "AvgTempDest")\
                   .withColumnRenamed("WeatherCode", "WeatherCodeDest")\
                   .withColumnRenamed("AvgWindSpeed", "AvgWindSpeedDest")


<br>


Checking for missing variables keeping in mind that the Nulls will be kept to keep the data that is significant



In [9]:
for column in joinedDF.columns:
    print(f"COL:\t {column}",
          f"NA:\t {joinedDF.where(F.col(column) == 'NA').count()}",
          f"NULL:\t {joinedDF.where(F.col(column).isNull()).count()}",
          f"M:\t {joinedDF.where(F.col(column) == 'M').count()}",
          f"T:\t {joinedDF.where(F.col(column) == '  T').count()}",
          f"' ':\t {joinedDF.where(F.col(column) == ' ').count()}",
          "",
          sep="\n"
         )
display(Markdown("This DataFrame has **%d rows**." % joinedDF.count()))


COL:	 Month
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 DayofMonth
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 DayOfWeek
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 ArrDelay
NA:	 1302
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 DepDelay
NA:	 1142
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 Origin
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 Dest
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 Cancelled
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 Diverted
NA:	 0
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 WeatherDelay
NA:	 80371
NULL:	 0
M:	 0
T:	 0
' ':	 0

COL:	 AvgTempOrigin
NA:	 0
NULL:	 23851
M:	 1202
T:	 0
' ':	 0

COL:	 AvgWindSpeedOrigin
NA:	 0
NULL:	 23851
M:	 468
T:	 0
' ':	 0

COL:	 WeatherCodeOrigin
NA:	 0
NULL:	 23851
M:	 0
T:	 0
' ':	 35243

COL:	 AvgTempDest
NA:	 0
NULL:	 23851
M:	 1201
T:	 0
' ':	 0

COL:	 AvgWindSpeedDest
NA:	 0
NULL:	 23851
M:	 464
T:	 0
' ':	 0

COL:	 WeatherCodeDest
NA:	 0
NULL:	 23851
M:	 0
T:	 0
' ':	 35253



This DataFrame has **100000 rows**.

In [10]:
joinedDF = joinedDF.filter((F.col("AvgTempOrigin") != "M") & (F.col("AvgWindSpeedOrigin") != "M"))
display(Markdown("This DataFrame has **%d rows**." % joinedDF.count()))

This DataFrame has **74651 rows**.

<br>

### ANALYZE DATA

In [11]:
joinedDF.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- AvgTempOrigin: string (nullable = true)
 |-- AvgWindSpeedOrigin: string (nullable = true)
 |-- WeatherCodeOrigin: string (nullable = true)
 |-- AvgTempDest: string (nullable = true)
 |-- AvgWindSpeedDest: string (nullable = true)
 |-- WeatherCodeDest: string (nullable = true)



In [56]:
print("\nSummary of columns:\n")

joinedDF.groupby("WeatherCodeOrigin").count().sort("count", ascending=False).show()
joinedDF.groupby("Cancelled").count().show()
joinedDF.groupby("Diverted").count().show()

joinedDF.select(["ArrDelay", "DepDelay", "WeatherDelay",
                 "AvgTempOrigin", "AvgWindSpeedOrigin"]
               ).summary().toPandas()


Summary of columns:

+-----------------+-----+
|WeatherCodeOrigin|count|
+-----------------+-----+
|                 |34786|
|            RA BR| 7995|
|               RA| 6044|
|               BR| 2723|
|            SN BR| 2689|
|               SN| 2445|
|            BR HZ| 1902|
|         RA DZ BR| 1415|
|         RA SN BR| 1212|
|         SN BR HZ|  788|
|         RA BR HZ|  786|
|               HZ|  778|
|        FG+ FG BR|  520|
|       SN FZFG BR|  498|
|           VCBLDU|  475|
|  RA DZ FG+ FG BR|  373|
|           FG+ BR|  367|
|            DZ BR|  319|
|       BCFG BR HZ|  302|
|      RA SN BR HZ|  301|
+-----------------+-----+
only showing top 20 rows

+---------+-----+
|Cancelled|count|
+---------+-----+
|        0|73911|
|        1|  740|
+---------+-----+

+--------+-----+
|Diverted|count|
+--------+-----+
|       0|74542|
|       1|  109|
+--------+-----+



,summary,ArrDelay,DepDelay,WeatherDelay,AvgTempOrigin,AvgWindSpeedOrigin
0,count,74651,74651,74651,74651,74651
1,mean,4.553494485244302,9.192082369336093,2.372763117986048,42.590105959732625,7.833446303464566
2,stddev,30.680121216484558,27.8865164571767,15.036412154883976,14.134081241524779,4.251099808477925
3,min,-1,-1,0,0,0.6
4,25%,-10.0,-2.0,0.0,33.0,4.5
5,50%,-3.0,0.0,0.0,45.0,7.2
6,75%,8.0,8.0,0.0,53.0,10.5
7,max,NA,NA,NA,9,9.9


***

<div class="alert alert-block alert-info">

# Question 1:

**Is there a relation between extreme conditions and the arrival delay?**
    
</div>

We will be using the below four dataset for our analysis:

<ul>
<li>extremeConditionsOrigin</li>
<li>extremeConditionsDest</li>
<li>notextremeConditionsOrigin</li>
<li>notextremeConditionsDest</li>
</ul>

<br>
They were filtered from the main joinedDF by taking into consideration that a Temperature lower than 32 Fahrenheit means 0 Degrees Celsius which is Freezing point. Moreover, most planes are delayed when there is strong wind which is above 30 miles/hr. The last variable which is WeatherCode, has extreme weather conditions in it like: Rain, Fog, Snow, Thunderstorm, ICe, Hail and many more. So when this variable is not empty means that there is no extreme weather conditions and vice versa.

In [13]:
exteremeConditionsOrigin = joinedDF.where( "AvgTempOrigin < 32 \
                                            or AvgWindSpeedOrigin > 30\
                                            or WeatherCodeOrigin not like ' '\
                                            ")


notexteremeConditionsOrigin = joinedDF.filter(F.col("WeatherCodeOrigin") == " ")\
                                      .where(" AvgTempOrigin > 32 or AvgWindSpeedOrigin < 30 ")



exteremeConditionsDest = joinedDF.where("AvgTempDest < 32\
                                        or AvgWindSpeedDest > 30\
                                        or WeatherCodeDest not like ' '\
                                        ")


notexteremeConditionsDest = joinedDF.filter(F.col("WeatherCodeDest") == " ")\
                                    .where(" AvgTempDest > 32 or AvgWindSpeedDest < 30 ")


exteremeConditionsOrigin.cache()
notexteremeConditionsOrigin.cache()
exteremeConditionsDest.cache()
notexteremeConditionsDest.cache()

DataFrame[Month: string, DayofMonth: string, DayOfWeek: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Cancelled: string, Diverted: string, WeatherDelay: string, AvgTempOrigin: string, AvgWindSpeedOrigin: string, WeatherCodeOrigin: string, AvgTempDest: string, AvgWindSpeedDest: string, WeatherCodeDest: string]

<br>

Let's look at the summaries:

In [14]:
print ("\nSummary of ArrDelay for extreme conditions:")
exteremeConditionsDest.select("ArrDelay").summary().show()


print ("\nSummary of ArrDelay for not extreme conditions:")
notexteremeConditionsDest.select("ArrDelay").summary().show()


Summary of ArrDelay for extreme conditions:
+-------+-----------------+
|summary|         ArrDelay|
+-------+-----------------+
|  count|            47264|
|   mean| 6.84933739331818|
| stddev|32.99055041671269|
|    min|               -1|
|    25%|             -9.0|
|    50%|             -2.0|
|    75%|             10.0|
|    max|               NA|
+-------+-----------------+


Summary of ArrDelay for not extreme conditions:
+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|             34505|
|   mean|0.6708705552636188|
| stddev| 26.17222043149078|
|    min|                -1|
|    25%|             -11.0|
|    50%|              -5.0|
|    75%|               4.0|
|    max|                NA|
+-------+------------------+



<br>

Comparing the Average Arrival Delay in extreme vs not extreme conditions in the `Origin` Airport:



In [15]:
AverageofArrDelayExtreme = exteremeConditionsOrigin.agg(F.avg("ArrDelay")).first()
AverageofArrDelayNotExtreme = notexteremeConditionsOrigin.agg(F.avg("ArrDelay")).first()

print("")
display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("Average Arrival Delay with Extreme Conditions",
       "Average Arrival Delay with No Extreme Conditions",
       "%s " % (AverageofArrDelayExtreme),
       "%s " % (AverageofArrDelayNotExtreme))))


| Average Arrival Delay with Extreme Conditions | Average Arrival Delay with No Extreme Conditions |
|----|----|
| 6.6136690493056  | 0.5496871016340248  |


<br>

Comparing the Average Arrival Delay in extreme vs not extreme conditions in the `Destination` Airport:



In [16]:
AverageofArrDelayExtreme1 = exteremeConditionsDest.agg(F.avg("ArrDelay")).first()
AverageofArrDelayNotExtreme1 = notexteremeConditionsDest.agg(F.avg("ArrDelay")).first()

display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("Average Arrival Delay with Extreme Conditions",
       "Average Arrival Delay with No Extreme Conditions",
       "%s " % (AverageofArrDelayExtreme1),
       "%s " % (AverageofArrDelayNotExtreme1))))


| Average Arrival Delay with Extreme Conditions | Average Arrival Delay with No Extreme Conditions |
|----|----|
| 6.84933739331818  | 0.6708705552636188  |


<br>

Looking at the two tables above, we notice that the arrival delay time is much more higher when we have extreme weather conditions. The difference from both Origin and Destination airport with  and without extreme conditions is around 6 minutes which means that weather is a really important variable that affects the arrival time of airplanes. 
Moreover, we can notice that the average delay in destination is 0.20 minutes more than arrival delay which is not that different. So, both weathers in Destination and Origin have a similar time influence on Arrival delay. 

***

<div class="alert alert-block alert-info">

# Question 2:
**Answer again the same question but focusing on the `WeatherDelay`.**
    
</div>

First, let's remove NAs in WeatherDelay

In [17]:
# filtering NAs in WeatherDelay

exteremeConditionsOrigin1 = exteremeConditionsOrigin.where(F.col("WeatherDelay")!="NA")
notexteremeConditionsOrigin1 = notexteremeConditionsOrigin.where(F.col("WeatherDelay")!="NA")

exteremeConditionsDest1 = exteremeConditionsDest.where(F.col("WeatherDelay")!="NA")
notexteremeConditionsDest1 = notexteremeConditionsDest.where(F.col("WeatherDelay")!="NA")


<br>

Comparing the Average Weather Delay in extreme vs not extreme conditions in the `Origin` Airport:



In [53]:
AverageofWeatherDelayExtreme = exteremeConditionsOrigin1.agg(F.avg("WeatherDelay")).first()
AverageofWeatherDelayNotExtreme = notexteremeConditionsOrigin1.agg(F.avg("WeatherDelay")).first()


display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("Average Weather Delay with Extreme Conditions",
       "Average Weather Delay with No Extreme Conditions",
       "%s " % (AverageofWeatherDelayExtreme),
       "%s " % (AverageofWeatherDelayNotExtreme))))


| Average Weather Delay with Extreme Conditions | Average Weather Delay with No Extreme Conditions |
|----|----|
| 2.7464744264365395  | 1.5773743635156077  |



Comparing the Average Weather Delay in extreme vs not extreme conditions in the `Destination` Airport:



In [54]:
AverageofWeatherDelayExtreme1 = exteremeConditionsDest1.agg(F.avg("WeatherDelay")).first()
AverageofWeatherDelayNotExtreme1 = notexteremeConditionsDest1.agg(F.avg("WeatherDelay")).first()


display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("Average Weather Delay with Extreme Conditions",
       "Average Weather Delay with No Extreme Conditions",
       "%s " % (AverageofWeatherDelayExtreme1),
       "%s " % (AverageofWeatherDelayNotExtreme1))))


| Average Weather Delay with Extreme Conditions | Average Weather Delay with No Extreme Conditions |
|----|----|
| 2.9031987414787626  | 1.0075839653304441  |


<br>

As we notice from both tables, the average weather delay with no extreme conditions is 1.58 and 1 minutes respectively which is a bit high. After doing some research we found that most airlines sometimes say that they are delayed because of the weather which is not always true since they would have other problems of delay because of the variables that were filtered out at the beginning of our analysis like: Carrier, NAS, Security or Late Aircraft delay. 

We found that there is a relation between extreme conditions and weather delay since we noticed an increase of 1.2 and 1.9 minutes respectively. The weather at the destination has a bit of a more effect on the weather delay.

As we can see from the previous summary of variables, we can see that the Weather Delay has a mean of 2.37 minutes which means that in both cases the delay is affected by the extreme conditions.


***
<div class="alert alert-block alert-info">
    
# Question 3:
**Is this relation constant across airports, or are there airports better prepared for extreme weather than others? Do you think there is a relation with the economic development of the city, or even with the location of the city?**

</div>

Preparing Data

In [20]:
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))

AirportOrigin = exteremeConditionsOrigin.groupBy("Origin")\
                                        .agg(F.avg("ArrDelay").alias("AverageArrivalDelay"),
                                             cnt_cond(F.col("AvgTempOrigin") < 32).alias("Count of Temperature"),
                                             cnt_cond(F.col("AvgWindSpeedOrigin") > 30).alias("Count of High Wind Speed"),
                                             cnt_cond(F.col("WeatherCodeOrigin") != " ").alias("Count of Extreme Weather"),
                                             F.count("Origin").alias("TtlFltsInExtConds"))


AirportOrigin = AirportOrigin.withColumn("TtlCntOfExtWeatConds",
                                         (F.col("Count of Temperature") +
                                          F.col("Count of High Wind Speed") +
                                          F.col("Count of Extreme Weather"))\
                                         .alias("TtlCntOfExtWeatConds"))


TotalFlightsperAirportsOrigin = joinedDF.groupBy("Origin").count()\
                                        .withColumnRenamed("count", "TotalNumberOfFlights")


JoinedAirportDF = AirportOrigin.join(TotalFlightsperAirportsOrigin, on = "Origin")\
                                .drop("Count of Temperature")\
                                .drop("Count of High Wind Speed")\
                                .drop("Count of Extreme Weather")    

JoinedAirportDF.cache()

DataFrame[Origin: string, AverageArrivalDelay: double, TtlFltsInExtConds: bigint, TtlCntOfExtWeatConds: bigint, TotalNumberOfFlights: bigint]

<br>

Top 20 origin airports with the `most` average delay:

In [21]:
JoinedAirportDF.orderBy(F.desc("AverageArrivalDelay"))\
               .select("Origin",
                       F.round("AverageArrivalDelay", 2).alias("AvgArrDelay"),
                       "TtlCntOfExtWeatConds",
                       "TtlFltsInExtConds",
                       "TotalNumberOfFlights").show()

+------+-----------+--------------------+-----------------+--------------------+
|Origin|AvgArrDelay|TtlCntOfExtWeatConds|TtlFltsInExtConds|TotalNumberOfFlights|
+------+-----------+--------------------+-----------------+--------------------+
|   LCH|       60.0|                   1|                1|                   1|
|   TUS|      47.14|                  28|               28|                 663|
|   SFO|      30.53|                 385|              385|                 515|
|   LAS|      26.25|                1386|             1386|                5162|
|   PHX|      16.81|                 585|              585|                4211|
|   DAY|       14.0|                   1|                1|                   1|
|   LAX|      13.88|                1508|             1508|                2430|
|   BUR|      13.09|                 641|              641|                1263|
|   DAL|      12.64|                1277|             1277|                2760|
|   SLC|      11.41|        

<br>

Top 20 origin airports with the `least` average delay:

In [22]:
JoinedAirportDF.orderBy(F.asc("AverageArrivalDelay"))\
               .select("Origin",
                       F.round("AverageArrivalDelay", 2).alias("AvgArrDelay"),
                       "TtlCntOfExtWeatConds",
                       "TtlFltsInExtConds",
                       "TotalNumberOfFlights").show()

+------+-----------+--------------------+-----------------+--------------------+
|Origin|AvgArrDelay|TtlCntOfExtWeatConds|TtlFltsInExtConds|TotalNumberOfFlights|
+------+-----------+--------------------+-----------------+--------------------+
|   LFT|       -7.0|                   1|                1|                   1|
|   BTR|       -4.0|                   1|                1|                   1|
|   ISP|      -3.44|                 608|              497|                 606|
|   BDL|      -3.09|                 452|              363|                 382|
|   AMA|      -2.95|                 140|              116|                 225|
|   ALB|      -1.26|                 421|              273|                 299|
|   MCO|       0.26|                1233|             1233|                2393|
|   RSW|       0.69|                 121|              121|                 242|
|   ORF|       0.87|                 189|              164|                 275|
|   RDU|       0.91|        

<br>

Ratios of Flights with most average delay:

In [23]:

JoinedAirportDF.withColumn("RatioOfExtremeOverTotalFlights",
                           F.col("TtlFltsInExtConds") / F.col("TotalNumberOfFlights"))\
               .withColumn("RatioOfExtremeWeatherPerFlight",\
                           F.col("TtlCntOfExtWeatConds") / F.col("TtlFltsInExtConds"))\
               .select("Origin",
                       F.round("AverageArrivalDelay", 2).alias("AvgArrDelay"),
                       F.round("RatioOfExtremeWeatherPerFlight", 2).alias("RatioOfExtremeWeatherPerFlight"),
                       F.round("RatioOfExtremeOverTotalFlights", 2).alias("RatioOfExtremeOverTotalFlights"))\
               .orderBy(F.desc("AverageArrivalDelay"))\
               .show()

+------+-----------+------------------------------+------------------------------+
|Origin|AvgArrDelay|RatioOfExtremeWeatherPerFlight|RatioOfExtremeOverTotalFlights|
+------+-----------+------------------------------+------------------------------+
|   LCH|       60.0|                           1.0|                           1.0|
|   TUS|      47.14|                           1.0|                          0.04|
|   SFO|      30.53|                           1.0|                          0.75|
|   LAS|      26.25|                           1.0|                          0.27|
|   PHX|      16.81|                           1.0|                          0.14|
|   DAY|       14.0|                           1.0|                           1.0|
|   LAX|      13.88|                           1.0|                          0.62|
|   BUR|      13.09|                           1.0|                          0.51|
|   DAL|      12.64|                           1.0|                          0.46|
|   

<br>

Ratios of Flights with least average delay:

In [24]:
JoinedAirportDF.withColumn("RatioOfExtremeOverTotalFlights",
                           F.col("TtlFltsInExtConds") / F.col("TotalNumberOfFlights"))\
               .withColumn("RatioOfExtremeWeatherPerFlight",
                               F.col("TtlCntOfExtWeatConds") / F.col("TtlFltsInExtConds"))\
               .select("Origin",
                       F.round("AverageArrivalDelay",2).alias("AvgArrDelay"),
                       F.round("RatioOfExtremeWeatherPerFlight",2).alias("RatioOfExtremeWeatherPerFlight"),
                       F.round("RatioOfExtremeOverTotalFlights",2).alias("RatioOfExtremeOverTotalFlights"))\
               .orderBy(F.asc("AverageArrivalDelay"))\
               .show()

+------+-----------+------------------------------+------------------------------+
|Origin|AvgArrDelay|RatioOfExtremeWeatherPerFlight|RatioOfExtremeOverTotalFlights|
+------+-----------+------------------------------+------------------------------+
|   LFT|       -7.0|                           1.0|                           1.0|
|   BTR|       -4.0|                           1.0|                           1.0|
|   ISP|      -3.44|                          1.22|                          0.82|
|   BDL|      -3.09|                          1.25|                          0.95|
|   AMA|      -2.95|                          1.21|                          0.52|
|   ALB|      -1.26|                          1.54|                          0.91|
|   MCO|       0.26|                           1.0|                          0.52|
|   RSW|       0.69|                           1.0|                           0.5|
|   ORF|       0.87|                          1.15|                           0.6|
|   

<br>

As we can notice from the first table with the most average delays, when extreme weather conditions happen in TUS (Tucson Internation airport), the average delay is around 47 minutes which is alot since this shows that they are not fully prepared for bad weather conditions since only 28 out of 663  flights got delayed due to this condition.

We can also see SFO (San Francisco Internation Airport) which is known for its weather delays, has lots of airplanes that fly under bad weather conditions 385/515 flights with an average delay of 30 minutes.

On the other hand, we can notice from the second table that ISP (Long Island MacArthur Airport) which is located in New York is fully equipped for bad weather conditions since it has 497/606 planes fly under extreme weather conditions but with a negative Average Delay. 

The same goes for BDL (Bradley International Airport) 363/382 flights with a negative average delay. Moreover, both airports face extreme weather conditions like low Temperature, high wind or even the WeatherCode variable (which is snow, fog, dust etc...) more than one time per flight.
Both airports are known for there professional services and up-to-date maintenance.

Finally, looking at the ratios we can notice that it does not matter how many extreme weather conditions or how many flights we have, but it matters on the airport if they are equipped or not for these conditions

***

<div class="alert alert-block alert-info">
    
# Question 4:
**Does `WeatherDelay` always entail a very bad arrival delay?**
    
</div>

In [25]:
print("\nSummary of ArrDelay for WeatherDelay > 0:")
weatherDelay = joinedDF.where("WeatherDelay > 0")
weatherDelay.select("ArrDelay").summary().show()


print("\nSummary of ArrDelay for WeatherDelay = 0:")
noWeatherDelay = joinedDF.where("WeatherDelay == 0")
noWeatherDelay.select("ArrDelay").summary().show()


weatherArrDelay75 = weatherDelay.agg(F.expr("percentile_approx(ArrDelay, 0.75)")).first()
weatherArrDelayaverage = weatherDelay.agg(F.avg("ArrDelay")).first()

noweatherArrDelay75 = noWeatherDelay.agg(F.expr("percentile_approx(ArrDelay, 0.75)")).first()
noweatherArrDelayaverage = noWeatherDelay.agg(F.avg("ArrDelay")).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("WeatherArrDelayAverage",
       "weatherArrDelay75%",
       "NoWeatherArrDelayAverage",
       "NoWeatherArrDelay75%",
       "%s " % (weatherArrDelayaverage),
       "%s " % (weatherArrDelay75),
       "%s " % (noweatherArrDelayaverage),
       "%s " % (noweatherArrDelay75))))



Summary of ArrDelay for WeatherDelay > 0:
+-------+-----------------+
|summary|         ArrDelay|
+-------+-----------------+
|  count|              729|
|   mean|92.73388203017832|
| stddev| 72.8001589962664|
|    min|              100|
|    25%|             37.0|
|    50%|             71.0|
|    75%|            128.0|
|    max|               99|
+-------+-----------------+


Summary of ArrDelay for WeatherDelay = 0:
+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|             12459|
|   mean| 47.79645236375311|
| stddev|44.943854490299614|
|    min|               100|
|    25%|              21.0|
|    50%|              31.0|
|    75%|              55.0|
|    max|                99|
+-------+------------------+




| WeatherArrDelayAverage | weatherArrDelay75% | NoWeatherArrDelayAverage | NoWeatherArrDelay75% |
|----|----|----|----|
| 92.73388203017832  | 128.0  | 47.79645236375311  | 55.0  |


<br>

We can see a difference of 45 minutes between the average arrival delay with and without having weather delays. Keep in mind that the 47 minutes (noweatherdelay) is the average of arrival delay with all variables included which means that we can have delays because of Carrier, NAS, Security or Late Aircraft delay.

Moreover, looking at the 75% percentile we notice a very huge difference of 73 minutes which confirms that the weather delay has a high impact on the Arrival Delay and entails always a bad delay.


***
<div class="alert alert-block alert-info">

# Question 5:
**What's the ratio of cancelled flights for each of the weather conditions at each origin airport?**
    
</div>

<br>

### I - Freezing

Prepare DF

In [26]:
freezingDF = joinedDF.where(F.col("AvgTempOrigin") <32)\
                     .groupBy("Origin").count()\
                     .withColumnRenamed("count", "CountTotal")

cancelledFreezingDF = joinedDF.where((F.col("Cancelled") == 1))\
                               .groupBy("Origin").count()\
                               .withColumnRenamed("count", "CountCancelled")

joinFreezingDF = freezingDF.join(cancelledFreezingDF, on="Origin")

<br>

Airports with the `lowest ratio` of flights cancelled in freezing conditions:

In [27]:
joinFreezingDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
              .select("Origin",
                      "CountTotal",
                      "CountCancelled",
                      F.round("Ratio",5).alias("RoundedRatio"))\
              .orderBy(F.col("RoundedRatio").asc())\
              .show()

+------+----------+--------------+------------+
|Origin|CountTotal|CountCancelled|RoundedRatio|
+------+----------+--------------+------------+
|   DTW|       297|             1|     0.00337|
|   DEN|       818|             3|     0.00367|
|   IND|       267|             1|     0.00375|
|   ALB|       218|             1|     0.00459|
|   SDF|       199|             1|     0.00503|
|   CMH|       386|             2|     0.00518|
|   OKC|       172|             1|     0.00581|
|   CLE|       261|             2|     0.00766|
|   SLC|       856|             7|     0.00818|
|   BDL|       235|             2|     0.00851|
|   MCI|      1139|            12|     0.01054|
|   COS|        94|             1|     0.01064|
|   STL|       927|            11|     0.01187|
|   BNA|       474|             6|     0.01266|
|   ABQ|       640|            10|     0.01563|
|   BWI|       961|            16|     0.01665|
|   MHT|       475|             8|     0.01684|
|   PIT|       218|             4|     0

Airports with the `highest ratio` of flights cancelled in freezing conditions:

In [28]:
joinFreezingDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
              .select("Origin",
                      "CountTotal",
                      "CountCancelled",
                      F.round("Ratio",5).alias("RoundedRatio"))\
              .orderBy(F.col("RoundedRatio").desc())\
              .show()

+------+----------+--------------+------------+
|Origin|CountTotal|CountCancelled|RoundedRatio|
+------+----------+--------------+------------+
|   MAF|        19|             1|     0.05263|
|   BUF|       232|            12|     0.05172|
|   ISP|       220|             9|     0.04091|
|   PHL|       345|            13|     0.03768|
|   GEG|       391|            14|     0.03581|
|   RNO|       450|            12|     0.02667|
|   ORF|        38|             1|     0.02632|
|   RDU|       115|             3|     0.02609|
|   AMA|        80|             2|       0.025|
|   PVD|       326|             7|     0.02147|
|   IAD|        96|             2|     0.02083|
|   MDW|      3729|            77|     0.02065|
|   LIT|        50|             1|        0.02|
|   BOI|       326|             6|      0.0184|
|   PIT|       218|             4|     0.01835|
|   MHT|       475|             8|     0.01684|
|   BWI|       961|            16|     0.01665|
|   ABQ|       640|            10|     0


### II - Windy

In [29]:
windspeedDF = joinedDF.where(F.col("AvgWindSpeedOrigin") >30)\
                      .groupBy("Origin").count()\
                      .withColumnRenamed("count", "CountTotal")

cancelledwindspeedDF = joinedDF.where((F.col("Cancelled") == 1))\
                                .groupBy("Origin").count()\
                                .withColumnRenamed("count", "CountCancelled")

joinWindspeedDF = windspeedDF.join(cancelledwindspeedDF, on="Origin")

<br>

Origin airports with the `highest ratio` of flights cancelled in conditions of extreme windspeed:



In [30]:
joinWindspeedDF.withColumn("Ratio", F.col("CountCancelled")/F.col("CountTotal"))\
               .select("Origin",
                       "CountTotal",
                       "CountCancelled",
                       F.round("Ratio",5).alias("RoundedRatio"))\
               .orderBy(F.col("RoundedRatio").desc())\
               .show()

+------+----------+--------------+------------+
|Origin|CountTotal|CountCancelled|RoundedRatio|
+------+----------+--------------+------------+
|   BUF|        17|            12|     0.70588|
+------+----------+--------------+------------+




### III - Other Extreme Conditions

Prepare DF

In [31]:
WeatherCodeDF = joinedDF.where(F.col("WeatherCodeOrigin") != " ")\
                        .groupBy("Origin").count()\
                        .withColumnRenamed("count", "CountTotal")


cancelledWeatherCodeDF = joinedDF.where((F.col("Cancelled") == 1))\
                                  .groupBy("Origin").count()\
                                  .withColumnRenamed("count", "CountCancelled")


joinWeatherCodeDF = WeatherCodeDF.join(cancelledWeatherCodeDF, on="Origin")

joinWeatherCodeDF.cache()

DataFrame[Origin: string, CountTotal: bigint, CountCancelled: bigint]

<br>

Origin airports with the `lowest ratio` of flights cancelled in summarized extreme weather conditions:

In [32]:
joinWeatherCodeDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
                 .select("Origin",
                         "CountTotal",
                         "CountCancelled",
                         F.round("Ratio",5).alias("RoundedRatio"))\
                 .orderBy(F.col("RoundedRatio").asc())\
                 .show()

+------+----------+--------------+------------+
|Origin|CountTotal|CountCancelled|RoundedRatio|
+------+----------+--------------+------------+
|   MSY|       606|             1|     0.00165|
|   DTW|       373|             1|     0.00268|
|   SDF|       317|             1|     0.00315|
|   IND|       316|             1|     0.00316|
|   LIT|       213|             1|     0.00469|
|   AUS|       626|             3|     0.00479|
|   CMH|       416|             2|     0.00481|
|   ALB|       203|             1|     0.00493|
|   OKC|       203|             1|     0.00493|
|   SEA|       525|             3|     0.00571|
|   SNA|       471|             3|     0.00637|
|   ORF|       151|             1|     0.00662|
|   RDU|       422|             3|     0.00711|
|   CLE|       260|             2|     0.00769|
|   BNA|       751|             6|     0.00799|
|   PBI|       120|             1|     0.00833|
|   JAN|       119|             1|      0.0084|
|   BWI|      1868|            16|     0

Origin airports with the `highest ratio` of flights cancelled in summarized extreme weather conditions:

In [33]:
joinWeatherCodeDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
                 .select("Origin",
                         "CountTotal",
                         "CountCancelled",
                         F.round("Ratio",5).alias("RoundedRatio"))\
                 .orderBy(F.col("RoundedRatio").desc())\
                 .show()

+------+----------+--------------+------------+
|Origin|CountTotal|CountCancelled|RoundedRatio|
+------+----------+--------------+------------+
|   PHX|       585|            46|     0.07863|
|   SFO|       385|            24|     0.06234|
|   HRL|       164|             8|     0.04878|
|   GEG|       295|            14|     0.04746|
|   LAS|      1386|            62|     0.04473|
|   SAN|      1236|            53|     0.04288|
|   LAX|      1508|            63|     0.04178|
|   BUF|       315|            12|      0.0381|
|   TUS|        28|             1|     0.03571|
|   AMA|        60|             2|     0.03333|
|   BFL|        33|             1|      0.0303|
|   MHT|       270|             8|     0.02963|
|   OAK|      2200|            61|     0.02773|
|   RNO|       491|            12|     0.02444|
|   BUR|       641|            15|      0.0234|
|   ISP|       388|             9|      0.0232|
|   ABQ|       437|            10|     0.02288|
|   FAT|        92|             2|     0

After filtering on the different weather conditions: 
    I: temperatures below 0
    II: extreme wind
    III: other extreme weather conditions,
we composed for each of the conditions a new dataframe consisting the origin airport, the total flights under the specific weather circumstances per origin airport, the cancelled flights under the specific weather circumstances per origin airport and the ratio of the latter two columns.

I.
MAF (Midland International Airport) with 5.26% (1/19) and BUF (Buffalo Niagara International Airport) with 5.17% (12/232) of the flights which were supposed to fly while the temperature was below the zero-point were canceled. These airports have the highest cancellation rates, but taking into account that only 19 flights departed from MAF under these circumstances and one of them got called off this percentage does not seem that terrible. Together with the fact that the state Texas, where MAF is located, throughout the year is relatively warm and the area is not used to cold temperatures.

DTW (Detroit Metropolitan Wayne County Airport) with the lowest cancellation rate of 0.38% (1/297) handles flying under freezing point remarkably better that BUF while both are located in the North-Eastern point of the U.S, where cold temperatures are not sparse.

II.
The only causes with extreme wind conditions left from BUF, the airport mentioned in the previous part. With a cancellation ratio of more that 70% (12/17) it can be assumed that the airport (and maybe the entire U.S.) was not expecting or not acting up to predictions of high wind speeds in January 2008.

III.
Extreme conditions, disregarding cold and wind speed, in general had the highest cancellation effect on airports in the South-West of the U.S. Eight of the top 10 airports with the highest ratios are located in Los Angeles, Arizona and Texas. This can be a coincidence, but looking at the numbers in the overview either the airports werent enough prepared to anticipate on these turn on events.

***

<div class="alert alert-block alert-info">

# Question 6:
**Can you answer the same about each destination airport?**
    
</div>

### I- Freezing

Prepare DF

In [34]:
freezingDF = joinedDF.where(F.col("AvgTempDest") <32)\
                     .groupBy("Dest").count()\
                     .withColumnRenamed("count", "CountTotal")

cancelledFreezingDF = joinedDF.where((F.col("Cancelled") == 1))\
                              .groupBy("Dest").count()\
                              .withColumnRenamed("count", "CountCancelled")


joinFreezingDF = freezingDF.join(cancelledFreezingDF, ["Dest"])

<br>

Destination airports with the `lowest ratio` of flights cancelled in freezing conditions:

In [35]:
joinFreezingDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
              .select("Dest",
                      "CountTotal",
                      "CountCancelled",
                      F.round("Ratio",5).alias("RoundedRatio"))\
              .orderBy(F.col("RoundedRatio").asc())\
              .show()

+----+----------+--------------+------------+
|Dest|CountTotal|CountCancelled|RoundedRatio|
+----+----------+--------------+------------+
| OMA|       367|             1|     0.00272|
| DEN|       812|             3|     0.00369|
| ALB|       215|             1|     0.00465|
| OKC|       173|             1|     0.00578|
| MCI|      1145|             8|     0.00699|
| CLE|       257|             2|     0.00778|
| CMH|       383|             3|     0.00783|
| DTW|       296|             3|     0.01014|
| MHT|       469|             5|     0.01066|
| BNA|       463|             5|      0.0108|
| COS|        92|             1|     0.01087|
| SLC|       821|            10|     0.01218|
| TUL|       164|             2|      0.0122|
| BOI|       322|             4|     0.01242|
| ABQ|       624|             8|     0.01282|
| STL|       912|            15|     0.01645|
| RDU|       111|             2|     0.01802|
| RNO|       488|            10|     0.02049|
| IAD|        94|             2|  

Destination airports with the `highest ratio` of flights cancelled in freezing conditions:

In [36]:
joinFreezingDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
              .select("Dest",
                      "CountTotal",
                      "CountCancelled",
                      F.round("Ratio",5).alias("RoundedRatio"))\
              .orderBy(F.col("RoundedRatio").desc())\
              .show()

+----+----------+--------------+------------+
|Dest|CountTotal|CountCancelled|RoundedRatio|
+----+----------+--------------+------------+
| MAF|        19|             1|     0.05263|
| AMA|        81|             4|     0.04938|
| BUF|       230|            11|     0.04783|
| PHL|       332|            14|     0.04217|
| GEG|       366|            13|     0.03552|
| PIT|       215|             6|     0.02791|
| ISP|       214|             5|     0.02336|
| MDW|      3650|            83|     0.02274|
| BWI|       948|            21|     0.02215|
| IAD|        94|             2|     0.02128|
| RNO|       488|            10|     0.02049|
| RDU|       111|             2|     0.01802|
| STL|       912|            15|     0.01645|
| ABQ|       624|             8|     0.01282|
| BOI|       322|             4|     0.01242|
| TUL|       164|             2|      0.0122|
| SLC|       821|            10|     0.01218|
| COS|        92|             1|     0.01087|
| BNA|       463|             5|  

### II- Windy

Prepare DF

In [37]:
windspeedDF = joinedDF.where(F.col("AvgWindSpeedDest") >30)\
                      .groupBy("Dest").count()\
                      .withColumnRenamed("count", "CountTotal")

cancelledwindspeedDF = flightsDF.where((F.col("Cancelled") == 1))\
                                .groupBy("Dest").count()\
                                .withColumnRenamed("count", "CountCancelled")

joinWindspeedDF = windspeedDF.join(cancelledwindspeedDF, ["Dest"])

<br>

Destination airports with the `highest ratio` of flights cancelled in conditions of extreme windspeed:

In [38]:
joinWindspeedDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
               .select("Dest",
                       "CountTotal",
                       "CountCancelled",
                       F.round("Ratio",5).alias("RoundedRatio"))\
               .orderBy(F.col("RoundedRatio").desc())\
               .show()

+----+----------+--------------+------------+
|Dest|CountTotal|CountCancelled|RoundedRatio|
+----+----------+--------------+------------+
| BUF|        17|            11|     0.64706|
+----+----------+--------------+------------+



### III- Other Extreme Conditions

In [39]:
WeatherCodeDF = joinedDF.where(F.col("WeatherCodeDest") != " ")\
                        .groupBy("Dest").count()\
                        .withColumnRenamed("count", "CountTotal")


 
cancelledWeatherCodeDF = flightsDF.where((F.col("Cancelled") == 1))\
                                  .groupBy("Dest").count()\
                                  .withColumnRenamed("count", "CountCancelled")


joinWeatherCodeDF = WeatherCodeDF.join(cancelledWeatherCodeDF, ["Dest"])

<br>

Destination airports with the `lowest ratio` of flights cancelled in summarized extreme weather conditions:

In [40]:
joinWeatherCodeDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
                 .select("Dest",
                         "CountTotal",
                         "CountCancelled",
                         F.round("Ratio",5).alias("RoundedRatio"))\
                 .orderBy(F.col("RoundedRatio").asc())\
                 .show()

+----+----------+--------------+------------+
|Dest|CountTotal|CountCancelled|RoundedRatio|
+----+----------+--------------+------------+
| FLL|       426|             1|     0.00235|
| MSY|       593|             2|     0.00337|
| OMA|       289|             1|     0.00346|
| RDU|       414|             2|     0.00483|
| ALB|       200|             1|       0.005|
| PVD|       360|             2|     0.00556|
| AUS|       679|             4|     0.00589|
| SEA|       505|             4|     0.00792|
| PBI|       120|             1|     0.00833|
| BDL|       235|             2|     0.00851|
| RSW|       117|             1|     0.00855|
| CMH|       444|             4|     0.00901|
| OKC|       201|             2|     0.00995|
| CRP|        94|             1|     0.01064|
| BNA|       742|             8|     0.01078|
| CLE|       274|             3|     0.01095|
| SLC|       772|            10|     0.01295|
| TUL|       229|             3|      0.0131|
| MCI|       932|            13|  

Destination airports with the `highest ratio` of flights cancelled in summarized extreme weather conditions:

In [41]:
joinWeatherCodeDF.withColumn("Ratio", F.col("CountCancelled") / F.col("CountTotal"))\
                 .select("Dest",
                         "CountTotal",
                         "CountCancelled",
                         F.round("Ratio",5).alias("RoundedRatio"))\
                 .orderBy(F.col("RoundedRatio").desc())\
                 .show()

+----+----------+--------------+------------+
|Dest|CountTotal|CountCancelled|RoundedRatio|
+----+----------+--------------+------------+
| PHX|       559|            75|     0.13417|
| SFO|       429|            44|     0.10256|
| LGB|        69|             6|     0.08696|
| HRL|       176|            12|     0.06818|
| LAX|      1454|            97|     0.06671|
| BFL|        31|             2|     0.06452|
| TUS|        31|             2|     0.06452|
| LAS|      1327|            85|     0.06405|
| BUR|       610|            36|     0.05902|
| SAN|      1298|            76|     0.05855|
| AMA|        72|             4|     0.05556|
| MRY|        58|             3|     0.05172|
| GEG|       315|            15|     0.04762|
| ONT|       730|            32|     0.04384|
| RNO|       518|            22|     0.04247|
| OAK|      2266|            91|     0.04016|
| PHL|       616|            23|     0.03734|
| BUF|       333|            11|     0.03303|
| MDW|      3456|           113|  

Similar to the previous question we filtered on the different weather conditions: 
    I: temperatures below 0
    II: extreme wind
    III: other extreme weather conditions,
we composed for each of the conditions a new dataframe consisting the destination airport, the total flights under the
specific weather circumstances per origin airport, the cancelled flights under the specific weather circumstances per 
origin airport and the ratio of the latter two columns.

I.
Two of the least performing origin airports are again in the list of top three worst anticipating destination airports. MAF with a ratio of 5.26% (1/19) and BUF with 4.78% (11/230). This could lead to a conclusion that or these weather circumstances are unusual for these airports or they were not ready to anticipate their operations on the conditions.

DEN (Denver International Airport) belongs, just like in the order of origin aiports, in the top three best performing airports with a ratio of 0.037% (3/812). Comparing locations of the better anticipating airports, there was no common region on which we could base any assumption.

II.
Just like in the previous question, BUF is the only airport with flights, this time arriving, flying with extreme wind speed. With a ratio of 64.7% (11/17) this stresses the fact of problems with the wind in January 2008. Finding only these high wind speeds in flights from and to BUF indicate on local conditions.

III.
Again comparing the destination airports with the highest cancellation rates to the ones from the origin airports, we can see that, this time, ten out of ten aiports are located in the South-East of the U.S. This confirms that we can assume that these airports need to take better precautions regarding extreme weather.

MSY (Louis Armstrong New Orleans International Airport) with the lowest cancellation ratio (0.017%) as origin airport and second lowest ratio (0.038%) as destination airport let the state of the weather influence the continuity of its flights the least.        

***

<div class="alert alert-block alert-info">

# Question 7:
**What"s the relation between the weather at the origin airport and the cancellation code (if any), the origin delay, and the `WeatherDelay`?**
    
</div>

<br>

Top 20 origin airports with the `most` average `weather delay`:

In [42]:
exteremeConditionsOrigin.groupBy("Origin")\
                        .agg(
                            F.avg("AvgTempOrigin").alias("AverageTemperature"),
                            F.avg("AvgWindSpeedOrigin").alias("AverageWind"),
                            cnt_cond(F.col("WeatherCodeOrigin") != " ").alias("Count of Extreme Weather"),
                            F.avg("WeatherDelay").alias("AverageWeatherDelay"))\
                        .orderBy(F.desc("AverageWeatherDelay"))\
                        .select(
                            "Origin",
                            F.round("AverageTemperature", 4).alias("AverageTemperature"),
                            F.round("AverageWind", 4).alias("AverageWind"),
                            "Count of Extreme Weather",
                            F.round("AverageWeatherDelay",4).alias("AverageWeatherDelay"))\
                        .show()

+------+------------------+-----------+------------------------+-------------------+
|Origin|AverageTemperature|AverageWind|Count of Extreme Weather|AverageWeatherDelay|
+------+------------------+-----------+------------------------+-------------------+
|   IAD|           32.7558|       6.05|                      98|            17.0385|
|   LGB|           54.6471|     4.7221|                      68|            10.3333|
|   PBI|           69.9583|    10.7492|                     120|             8.0556|
|   CLE|           25.8012|    10.8943|                     260|             7.4426|
|   TUL|           33.0652|     10.086|                     233|             7.1509|
|   BUF|           27.0632|    12.9256|                     315|             6.5938|
|   SLC|           21.7727|     7.8196|                     764|             6.2731|
|   SJC|           47.7278|     5.8084|                    1286|             6.1991|
|   DTW|           26.1398|     10.712|                     373| 

Top 20 origin airports with the `most` average `departure delay`:

In [43]:
exteremeConditionsOrigin.groupBy("Origin")\
                        .agg(F.avg("AvgTempOrigin").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedOrigin").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeOrigin") != " ").alias("Count of Extreme Weather"),
                             F.avg("DepDelay").alias("AverageDepDelay"))\
                        .orderBy(F.desc("AverageDepDelay"))\
                        .select("Origin",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageDepDelay", 4).alias("AverageDepDelay"))\
                        .show()

+------+------------------+-----------+------------------------+---------------+
|Origin|AverageTemperature|AverageWind|Count of Extreme Weather|AverageDepDelay|
+------+------------------+-----------+------------------------+---------------+
|   TUS|              58.0|        6.3|                      28|           48.5|
|   SFO|           47.6104|     7.9057|                     385|        34.9917|
|   LAS|           47.4957|     8.6946|                    1386|        30.8209|
|   PHX|           55.6581|     6.6043|                     585|        23.9583|
|   LAX|           55.1903|     7.5441|                    1508|          17.86|
|   BUR|           51.5725|     7.0657|                     641|        15.2828|
|   SAN|            54.415|     5.2604|                    1236|         14.645|
|   HRL|           59.3415|    11.1927|                     164|        14.3974|
|   SLC|           21.7727|     7.8196|                     764|        14.0364|
|   DAL|           43.2647| 

Top 20 origin airports with the `least` average `weather delay`:

In [44]:
exteremeConditionsOrigin.groupBy("Origin")\
                        .agg(F.avg("AvgTempOrigin").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedOrigin").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeOrigin") != " ").alias("Count of Extreme Weather"),
                             F.avg("WeatherDelay").alias("AverageWeatherDelay"))\
                        .orderBy(F.asc("AverageWeatherDelay"))\
                        .select("Origin",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageWeatherDelay",4).alias("AverageWeatherDelay"))\
                        .show()

+------+------------------+-----------+------------------------+-------------------+
|Origin|AverageTemperature|AverageWind|Count of Extreme Weather|AverageWeatherDelay|
+------+------------------+-----------+------------------------+-------------------+
|   LFT|              59.0|        9.8|                       1|               null|
|   DAY|              38.0|       21.9|                       1|               null|
|   BTR|              39.0|        8.0|                       1|               null|
|   AMA|           31.4828|    15.1241|                      60|                0.0|
|   COS|             21.97|      8.745|                      53|                0.0|
|   CRP|           54.6413|    12.4413|                      92|                0.0|
|   ELP|              50.5|      12.05|                     144|                0.0|
|   LCH|              62.0|        8.2|                       1|                0.0|
|   OKC|           32.8095|    13.7171|                     203| 

Top 20 origin airports with the `least` average `departure delay`:

In [45]:
exteremeConditionsOrigin.groupBy("Origin")\
                        .agg(F.avg("AvgTempOrigin").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedOrigin").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeOrigin") != " ").alias("Count of Extreme Weather"),
                             F.avg("DepDelay").alias("AverageDepDelay"))\
                        .orderBy(F.asc("AverageDepDelay"))\
                        .select("Origin",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageDepDelay", 4).alias("AverageDepDelay"))\
                        .show()

+------+------------------+-----------+------------------------+---------------+
|Origin|AverageTemperature|AverageWind|Count of Extreme Weather|AverageDepDelay|
+------+------------------+-----------+------------------------+---------------+
|   LCH|              62.0|        8.2|                       1|          -10.0|
|   DAY|              38.0|       21.9|                       1|           -7.0|
|   BTR|              39.0|        8.0|                       1|           -4.0|
|   LFT|              59.0|        9.8|                       1|           -3.0|
|   COS|             21.97|      8.745|                      53|         1.5556|
|   AMA|           31.4828|    15.1241|                      60|         2.6053|
|   BDL|           28.5372|     6.1504|                     217|         3.3269|
|   ALB|           25.9194|      6.759|                     203|         3.7169|
|   BFL|           48.4848|     6.0576|                      33|         3.8125|
|   ORF|           40.1951| 

After merging and filtering the dataset, we did not have any Cancellation Code, they wear already a very small part of our data set which will not make any changes.

The  important variable from our joined dataset that have a **correlation** with the Average Weather Delay is Extreme Weather (Weather Code). Nearly most of the average wind and average temperature are less than the  extreme weather threshold meaning that they do not have any impact on the Delay.

COD (Yellowstone Regional Airport), BDL (Bradley International Airport) and ALB (Albany International Airport) are equipped with the necessary equipment to combat extreme weathers since they have low temperatures (lower than 0 degrees celsius) while being in the top 20 airports with lowest Departure Delay.

Moreover, the departure delay has also a **correlation** with the extreme weather variable, when we have one count of extreme weather we have a negative departure delay. Once the extreme weather cases increase, we see an increase in the Departure Delay.

***

<div class="alert alert-block alert-info">

# Question 8:
**What"s the relation between the weather at the destination airport, the arrival delay, and the `WeatherDelay`?**
    
</div>

<br>

Top 20 origin airports with the `most` average `weather delay`:

In [46]:
exteremeConditionsOrigin.groupBy("Dest")\
                        .agg(F.avg("AvgTempDest").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedDest").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeDest") != " ").alias("Count of Extreme Weather"),
                             F.avg("WeatherDelay").alias("AverageWeatherDelay"))\
                        .orderBy(F.desc("AverageWeatherDelay"))\
                        .select("Dest",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageWeatherDelay",4).alias("AverageWeatherDelay"))\
                        .show()

+----+------------------+-----------+------------------------+-------------------+
|Dest|AverageTemperature|AverageWind|Count of Extreme Weather|AverageWeatherDelay|
+----+------------------+-----------+------------------------+-------------------+
| FAT|           47.6949|     5.5814|                      51|               12.4|
| SFO|           48.0189|     8.3375|                     248|             8.3209|
| MDW|           20.6441|    11.1681|                    2475|              7.856|
| GEG|            22.289|     8.2938|                     218|             7.8475|
| HRL|           58.7927|    11.6497|                     157|              7.725|
| LAS|           46.0124|     6.1114|                    1007|              7.243|
| BOI|            28.273|     7.8133|                     219|             6.6818|
| PHL|           34.4746|     9.3925|                     416|             6.0628|
| BUF|           27.1783|    12.5613|                     225|             5.4359|
| SL

Top 20 origin airports with the `most` average `departure delay`:

In [47]:
exteremeConditionsOrigin.groupBy("Dest")\
                        .agg(F.avg("AvgTempDest").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedDest").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeDest") != " ").alias("Count of Extreme Weather"),
                             F.avg("DepDelay").alias("AverageDepDelay"))\
                        .orderBy(F.desc("AverageDepDelay"))\
                        .select("Dest",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageDepDelay", 4).alias("AverageDepDelay"))\
                        .show()

+----+------------------+-----------+------------------------+---------------+
|Dest|AverageTemperature|AverageWind|Count of Extreme Weather|AverageDepDelay|
+----+------------------+-----------+------------------------+---------------+
| SAV|              47.0|        8.1|                       0|           43.0|
| SFO|           48.0189|     8.3375|                     248|        40.2158|
| IAH|              52.9|      11.81|                       6|           29.6|
| BFL|           49.6667|      7.063|                      20|        18.9615|
| EUG|             34.75|     7.5188|                      13|        18.1875|
| MRY|           51.4651|     4.9442|                      37|        17.4419|
| SLC|            23.727|     9.1106|                     438|        16.7362|
| MAF|           43.3937|     9.9591|                      50|        15.0952|
| PHL|           34.4746|     9.3925|                     416|         14.803|
| LIT|           37.8519|     8.2148|               

Top 20 origin airports with the `least` average `weather delay`:

In [57]:
exteremeConditionsOrigin.groupBy("Dest")\
                        .agg(F.avg("AvgTempDest").alias("AverageTemperature"),
                             F.avg("AvgWindSpeedDest").alias("AverageWind"),
                             cnt_cond(F.col("WeatherCodeDest") != " ").alias("Count of Extreme Weather"),
                             F.avg("WeatherDelay").alias("AverageWeatherDelay"))\
                        .orderBy(F.asc("AverageWeatherDelay"))\
                        .select("Dest",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageWeatherDelay",4).alias("AverageWeatherDelay"))\
                        .show()

+----+------------------+-----------+------------------------+-------------------+
|Dest|AverageTemperature|AverageWind|Count of Extreme Weather|AverageWeatherDelay|
+----+------------------+-----------+------------------------+-------------------+
| GSO|              27.0|        5.8|                       0|               null|
| DCA|              38.0|        7.6|                       0|               null|
| XNA|              22.0|        6.6|                       1|               null|
| EUG|             34.75|     7.5188|                      13|                0.0|
| COS|              25.5|     8.8417|                      27|                0.0|
| SAV|              47.0|        8.1|                       0|                0.0|
| EWR|              32.5|      7.875|                       2|                0.0|
| IAH|              52.9|      11.81|                       6|                0.0|
| TUL|           35.3414|    10.4545|                     162|                0.0|
| MR

Top 20 origin airports with the `least` average `departure delay`:

In [58]:
exteremeConditionsOrigin.groupBy("Dest")\
                        .agg(
                            F.avg("AvgTempDest").alias("AverageTemperature"),
                            F.avg("AvgWindSpeedDest").alias("AverageWind"),
                            cnt_cond(F.col("WeatherCodeDest") != " ").alias("Count of Extreme Weather"),
                            F.avg("DepDelay").alias("AverageDepDelay"))\
                        .orderBy(F.asc("AverageDepDelay"))\
                        .select("Dest",
                                F.round("AverageTemperature", 4).alias("AverageTemperature"),
                                F.round("AverageWind", 4).alias("AverageWind"),
                                "Count of Extreme Weather",
                                F.round("AverageDepDelay", 4).alias("AverageDepDelay"))\
                        .show()

+----+------------------+-----------+------------------------+---------------+
|Dest|AverageTemperature|AverageWind|Count of Extreme Weather|AverageDepDelay|
+----+------------------+-----------+------------------------+---------------+
| DCA|              38.0|        7.6|                       0|           -2.0|
| COS|              25.5|     8.8417|                      27|         0.8592|
| XNA|              22.0|        6.6|                       1|            2.0|
| GSO|              27.0|        5.8|                       0|            2.0|
| EWR|              32.5|      7.875|                       2|            5.0|
| RDU|           39.6967|     5.9097|                     312|         6.3303|
| MCO|           63.6092|     8.2414|                     886|         7.0049|
| BWI|           33.1464|     6.1136|                    1295|          7.172|
| TPA|           63.3488|     7.5687|                     748|         7.3421|
| ORF|           40.8556|     7.9551|               

We can notice that same as our Origin Analysis, the  important variables from our joined dataset that have a high **correlation** with the AvgWeatherDelay are Extreme Weather (Weather Code) followed by Temperature. Nearly most of the average wind are less than 30 which means it is not dangerous.

We can notice in MDW (Chicago Midway International Airport) we have a temperature lower than 0 degrees and a lot of extreme weather conditions.But we can notice that its not in the top 20 airports with high Dep Delay but with is the 20th in lowest Dep Delay, which means that this airport is equipped for bad weathers or any other issue that will delay the plane. Same goes for GSO (Piedmont Triad International Airport), XNA (Northwest Arkansas Regional Airport) and  COS (Colorado Springs Airport) looking at the least Average Weather Delay.

Moreover, we can notice that the Average Departure Delay is **correlated** a bit to the extreme conditions but not to the temperature and wind. Therefore, other types of delays have to be taken into consideration as discussed before: Carrier, NAS, Security or Late Aircraft delays. 

## Conclusion

We can conclude that the weather conditions have a high correlation with Arrival, Departure and Weather Delay. Once we start having extreme conditions, we can see that all the Delays start to increase.

After doing some analysis, we can notice that some airports are much more equipped and ready to combat extreme weathers while others are still not developing or buying new equipments to decrease their Arrival and Departure Delays. 

We have to keep in mind that although weather is the most factor that affects delays, we also have Carrier, NAS, Security and Late Aircraft delays. 